In [8]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
import sklearn
from prettytable import PrettyTable

In [6]:
data = pd.read_csv('resources/train_EURUSD_90_M6.txt', names=np.hstack((np.arange(0, 90, 1), np.array(['up', 'down', 'itime', 'open']))))
data.head()

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,up,down,itime,open
0,0.91,0.22,0.14,0.04,0.0,0.73,0.23,0.41,0.03,0.03,...,0.29,0.71,0.53,1.37,0.63,0.51,0,1,1420189200,1.20538
1,0.91,0.27,0.14,0.05,0.0,0.73,0.23,0.41,0.07,0.01,...,0.29,0.72,0.53,1.37,0.62,0.51,0,1,1420189560,1.20412
2,0.89,0.34,0.14,0.08,0.0,0.71,0.28,0.41,0.05,0.01,...,0.29,0.73,0.53,1.37,0.67,0.51,0,1,1420189920,1.20399
3,0.87,0.37,0.14,0.09,0.0,0.70,0.28,0.41,0.04,0.01,...,0.29,0.72,0.53,1.37,0.64,0.51,0,1,1420190280,1.20430
4,0.85,0.37,0.14,0.06,0.0,0.68,0.26,0.41,0.09,0.01,...,0.29,0.72,0.53,1.37,0.66,0.51,0,1,1420190640,1.20470


In [7]:
values = data.drop(['up', 'down', 'itime', 'open'], 1)
labels = data[['up', 'down']]

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(values.values, labels.values, test_size=0.2, shuffle=False)

In [10]:
from tensorflow.python.keras.layers import Dense, Dropout

def create_model(input_dim, act, dropout, layers):
    model = tf.keras.Sequential()
    if layers == 1:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
    if layers == 2:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.5), activation=act))
        model.add(Dropout(dropout))
    if layers == 3:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.5), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.3), activation=act))
        model.add(Dropout(dropout))
    if layers == 4:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.7), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.5), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.3), activation=act))
        model.add(Dropout(dropout))
    if layers == 5:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.8), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.6), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.4), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.2), activation=act))
        model.add(Dropout(dropout))
    model.add(Dense(2, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model

def test_result(model, x_test, y_test):
    n = x_test.shape[0]
    a = np.empty(shape=[n, 1])
    pred = model.predict(x_test)
    table = PrettyTable()
    best_res = 0
    table.field_names = ['Answer', 'Win', 'Lose', 'Win (%)', 'Answer (%)']
    for i, pr in enumerate(pred): a[i] = int(round((pr[0] - pr[1]) * 100, 0))
    results = np.hstack([a, y_test])
    for lim in range(0, 99, 2):
        win = lose = 0
        for i, d in enumerate(results):
            if abs(d[0]) >= lim:
                if (d[0] > 0 and d[1] == 1) or (d[0] < 0 and d[1] == 0):
                    win += 1
                else:
                    lose += 1
        if win + lose > 0:
            res = int((win / (win + lose)) * 100)
        else:
            res = 0
        trades = round(((win + lose) / results.shape[0]) * 100, 1)
        if trades < 1: break
        # best_res = res
        # best_lim = lim
        # best_trades = trades
        table.add_row([lim, win, lose, res, trades])
    # if best_res <= n_lim:
    #     clear_output(wait=True)
    #     print("⚠️️ Результат <", n_lim, ". Очень низкое качество обучения." +
    #           "\nПовторите обучение c другими настройками нейросети.")
    #     print("Заново загружать данные не нужно.")
    # else:
    print(table)
    # print("\n✅️ Последнее значение -", str(best_res) + "% правильных ответов.\n" +
    #         'Этот результат будет получен если брать ответы с "уверенность" больше', best_lim, "(Ответ сети).",
    #         "\nСигналов будет -", str(best_trades) +
    #         "% от общего количества свечей которые подавались на тестирование.")
    # return best_lim, best_res, results

In [19]:
act = 'relu'
dropout = 0.2
epochs = 50
input_dim = values.shape[1]

model = create_model(input_dim, act, dropout, 4)
history = model.fit(x_train, y_train, epochs=epochs, validation_split=0.2, batch_size=128, verbose=1)

Epoch 1/50
1772/1772 [==============================] - 6s 3ms/step - loss: 0.6934 - accuracy: 0.5003 - val_loss: 0.6933 - val_accuracy: 0.4947

In [20]:
test_result(model, x_test, y_test)


+--------+-------+-------+---------+------------+
| Answer |  Win  |  Lose | Win (%) | Answer (%) |
+--------+-------+-------+---------+------------+
|   0    | 35120 | 35753 |    49   |   100.0    |
|   2    | 25261 | 24325 |    50   |    70.0    |
|   4    | 17080 | 16070 |    51   |    46.8    |
|   6    | 11529 | 10322 |    52   |    30.8    |
|   8    |  5916 |  4888 |    54   |    15.2    |
|   10   |  3670 |  2925 |    55   |    9.3     |
|   12   |  2642 |  2010 |    56   |    6.6     |
|   14   |  1986 |  1407 |    58   |    4.8     |
|   16   |  1480 |  997  |    59   |    3.5     |
|   18   |  1061 |  674  |    61   |    2.4     |
|   20   |  768  |  475  |    61   |    1.8     |
|   22   |  549  |  337  |    61   |    1.3     |
+--------+-------+-------+---------+------------+
